In [ ]:
import os
#os.environ['CUDA_VISIBLE_DEVICES']="1"
import torch
import numpy as np
import random
from fastprogress import master_bar, progress_bar
from fastai import *
from fastai.vision import *
#from fastai.callbacks import *

from fastai.distributed import *
from fastai.metrics import accuracy
import pandas as pd
import torch
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import torch.nn as nn
#import pretrainedmodels
import math
import sys
import pickle
#from arch import RingGeMNet, GeMNet, L2Norm, GeM

#from arch import GeM, L2Norm
%config InlineBackend.figure_format ='retina'

SEED = 42
np.random.seed(SEED), random.seed(SEED), torch.random.manual_seed(SEED), torch.cuda.manual_seed_all(SEED)

(None, None, <torch._C.Generator at 0x7bf0e41ec450>, None)

In [ ]:
import pandas as pd
from pathlib import Path

path = Path('../google-landmark-train/')
train_df = pd.read_csv(path / '/content/drive/MyDrive/Train/train.csv', usecols=[0,2], converters={'id':lambda x:x[0]+'/'+x[1]+'/'+x[2]+'/'+x})
train_df.head()
min_images, n_landmarks = 200, 1000
train_df = train_df.groupby('landmark_id').filter(lambda x: len(x) >= min_images)

In [ ]:
train_df = train_df[train_df['landmark_id'].isin(
    np.random.choice(train_df['landmark_id'].unique(), n_landmarks, replace=False))]

In [ ]:
train_df.head()

,id,landmark_id
1,2/0/2/202cd79556f30760,104169
9,8/d/f/8df019949b8db328,81049
11,c/c/f/ccf070a2da242d56,138982
23,0/5/3/053d1409647dfba2,74821
25,3/c/d/3cdc355c6232712f,12877


In [ ]:
train_df['target'] = 1
train1 = train_df.copy()
train1['target'] = 0
train_df = pd.concat([train_df, train1], ignore_index=True)


In [ ]:
train_df.head()

,id,landmark_id,target
0,2/0/2/202cd79556f30760,104169,1
1,8/d/f/8df019949b8db328,81049,1
2,c/c/f/ccf070a2da242d56,138982,1
3,0/5/3/053d1409647dfba2,74821,1
4,3/c/d/3cdc355c6232712f,12877,1


In [ ]:
from fastai.vision.core import TensorImage
from fastai.data.block import Transform
from fastai.data.block import DataBlock
from fastai.vision.data import ImageBlock, get_image_files
from fastai.vision.augment import Resize
from fastai.data.transforms import RandomSplitter
from pathlib import Path

class SiamImage:
    def __init__(self, img1, img2):
        self.img1, self.img2 = img1, img2

    def show(self, ctx=None, **kwargs):
        # Ghép hai ảnh cạnh nhau để hiển thị
        combined_img = torch.cat([self.img1, self.img2], dim=2)  # Ghép theo chiều rộng
        return TensorImage(combined_img).show(ctx=ctx, **kwargs)

class SiamTransform(Transform):
    def encodes(self, x):
        # Biến đổi dữ liệu thành đối tượng SiamImage
        return SiamImage(x[0], x[1])

# Thiết lập DataBlock với SiamTransform
from fastai.vision.data import ImageBlock

dblock = DataBlock(
    blocks=(ImageBlock, ImageBlock),
    get_items=get_image_files,  # Hàm lấy danh sách các file ảnh
    splitter=RandomSplitter(),
    item_tfms=[Resize(224)],  # Resize ảnh
    batch_tfms=[],
)

# Đọc dữ liệu
path = Path('/content/drive/MyDrive/Train')
dls = dblock.dataloaders(path)


In [ ]:
from fastai.vision.all import *
import numpy as np
import random
from pathlib import Path

# Định nghĩa lớp SiamImage để lưu trữ 2 ảnh
class SiamImage(Tuple):
    def show(self, ctx=None, **kwargs):
        img1, img2 = self
        if not isinstance(img1, TensorImage): img1 = PILImage.create(img1)
        if not isinstance(img2, TensorImage): img2 = PILImage.create(img2)
        t1 = img1.show(ctx=ctx, **kwargs)
        t2 = img2.show(ctx=ctx, **kwargs)
        return t2

# Hàm tạo cặp ảnh (positive và negative pair)
def make_siamese_pairs(files):
    """Tạo danh sách các cặp ảnh (similar và different)"""
    random.shuffle(files)
    pairs = []
    for file in files:
        # Chọn ảnh tương tự
        similar = random.choice([f for f in files if f.stem.split('_')[0] == file.stem.split('_')[0]])
        # Chọn ảnh khác
        different = random.choice([f for f in files if f.stem.split('_')[0] != file.stem.split('_')[0]])
        pairs.append((file, similar))  # Positive pair
        pairs.append((file, different))  # Negative pair
    return pairs

# Hàm gán nhãn: 1 nếu ảnh tương tự, 0 nếu khác
def label_func(pair):
    return 1 if pair[0].stem.split('_')[0] == pair[1].stem.split('_')[0] else 0

# Hàm chia tập dữ liệu train/valid
def siamese_splitter(items):
    """Trả về các chỉ mục để chia thành tập train/valid"""
    train_idxs = L(np.arange(len(items) // 2))
    valid_idxs = L(np.arange(len(items) // 2, len(items)))
    return train_idxs, valid_idxs

# Định nghĩa DataBlock
siamese = DataBlock(
    blocks=(ImageBlock, CategoryBlock),  # Một ảnh (cặp ảnh) và một nhãn
    get_items=get_image_files,  # Lấy danh sách file ảnh
    get_x=lambda pair: SiamImage(pair[0], pair[1]),  # Chuyển cặp ảnh thành đối tượng SiamImage
    get_y=label_func,  # Gán nhãn
    splitter=RandomSplitter(valid_pct=0.2),  # Chia train/valid
    item_tfms=Resize(224),  # Resize ảnh về 224x224
    batch_tfms=aug_transforms(mult=2.0)  # Augmentation
)

# Tạo cặp ảnh từ dữ liệu
path = Path('/content/drive/MyDrive/Train')  # Thay đường dẫn dữ liệu của bạn
files = get_image_files(path)
pairs = make_siamese_pairs(files)

# Tạo DataLoaders
dls = siamese.dataloaders(pairs, bs=16)

# Hiển thị một batch dữ liệu
dls.show_batch(max_n=9, figsize=(10, 10))


In [ ]:
arch=models.resnet152

In [ ]:
from fastai.vision.all import *
import torch.nn.functional as F

class SiameseNet(nn.Module):
    def __init__(self, arch):
        super(SiameseNet, self).__init__()
        # Sử dụng create_body để tạo phần trích xuất đặc trưng
        self.cnn = nn.Sequential(
            create_body(arch, cut=-2),  # Tạo phần body từ mô hình ResNet
            nn.AdaptiveAvgPool2d(1),   # Lấy trung bình thích ứng
            nn.Flatten(),              # Làm phẳng đầu ra thành vector
            L2Norm()                   # Chuẩn hóa L2
        )

    def forward(self, x1, x2):
        # Xử lý hai ảnh đầu vào qua mạng CNN
        output1 = self.cnn(x1)
        output2 = self.cnn(x2)
        return output1, output2


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ContrastiveLoss(nn.Module):
    """
    Contrastive loss for Siamese Networks.
    Takes embeddings of two samples and a target label:
    - target == 1 if samples are from the same class,
    - target == 0 otherwise.
    """
    def __init__(self, margin=5.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, ops, target):
        op1, op2 = ops[0], ops[1]

        # Calculate pairwise distance
        dist = F.pairwise_distance(op1, op2)

        # Positive and negative losses
        positive_loss = target * (dist ** 2)  # Target == 1
        negative_loss = (1 - target) * (F.relu(self.margin - dist) ** 2)  # Target == 0

        # Combine losses
        loss = 0.5 * (positive_loss + negative_loss)
        return loss.mean()


In [ ]:
# Định nghĩa mô hình SiameseNet với kiến trúc backbone
model = SiameseNet(arch)

# Tạo Learner
siam_learner = Learner(
    dls,  # DataLoader (DataBlock được khởi tạo trước đó)
    model,
    loss_func=ContrastiveLoss(margin=0.5),
    metrics=[]  # Thêm metric nếu cần (vd: accuracy, F1, ...)
)

# Sử dụng kiểu FP16 để tăng hiệu năng huấn luyện
siam_learner = siam_learner.to_fp16()

# Điều chỉnh batch size trong DataLoader (nếu cần)
dls.bs = 64  # Batch size

# Tìm Learning Rate tốt nhất
siam_learner.lr_find()
siam_learner.plot_lr_find()  # Thay cho siam_learner.recorder.plot()

# Huấn luyện mô hình
#siam_learner.fit_one_cycle(3, 1e-3)  # 3 epochs, với learning rate 1e-3


In [ ]:
data.train_ds[0][0].img1
data.train_ds[0][0].img1.data
siam_learner.to_fp32().model.cnn(data.train_ds[0][0].img1.data.unsqueeze(0).cuda())
model_name = f'{arch.__name__}_i{min_images}_l{n_landmarks}-{SZ}'
learn.save(model_name)
previous_model_name = model_name
previous_model_name